<a href="https://colab.research.google.com/github/faizankshaikh/ForaGym/blob/main/examples/trial1_foragymSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --quiet https://github.com/faizankshaikh/ForaGym.git
%cd ForaGym
!pip install -q -e .
!pip install stable-baselines3

In [1]:
import gym
import foragym
import numpy as np

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
env = gym.make("foragym:foragym/ForaGym-v0")

In [3]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    done = False
    state = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not done:
        action = 0 #env.action_space.sample()
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, done, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Alive?: {not done}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [0 1 1 0 0]
--Current life: 2
--Type of Weather: Rainy

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: True

Current game state:-
--Days left: 5
--State of Field: [1 0 1 0 1]
--Current life: 1
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: False

Current game state:-
--Days left: 4
--State of Field: [0 0 0 0 1]
--Current life: 0
--Type of Weather: Clear

----------
Episode #2
Initial state:-
--Days left: 6
--State of Field: [1 0 0 0 1]
--Current life: 3
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: True

Current game state:-
--Days left: 5
--State of Field: [0 0 1 1 0]
--Current life: 2
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: True

Current game state:-
--Days left: 4
--State of Field: [0 0 1 0 1]
--Current life: 1
--Type of Weather: Clear

Action to ta

In [4]:
num_episodes = 1000
episode_reward = []

for episode in range(num_episodes):
    done = False
    state = env.reset()

    total_reward = 0

    while not done:
        action = 0 #env.action_space.sample()
        state, reward, done, info = env.step(action)

        total_reward += reward

    episode_reward.append(total_reward)

print(f"mean_reward:{np.mean(episode_reward):.2f} +/- {np.std(episode_reward):.2f}")


mean_reward:-3.50 +/- 1.74


In [5]:
num_states = env.NUM_STATES
num_actions = env.NUM_ACTIONS

V = np.zeros(num_states)
optimal_policy = np.zeros([num_states, num_actions])

for s in range(num_states):
	actions_s = np.zeros(num_actions)
	for a in range(num_actions):
		for transition_prob, next_state, reward, done in env.P[s][a]:
			actions_s[a] += reward + transition_prob * V[next_state]
	V[s] = actions_s.max()
	optimal_policy[s, np.argmax(actions_s)] = 1.0


In [6]:
# for idx, pol in enumerate(optimal_policy):
#     field, life_point, weather = env.decode(idx)
#     print(f"field: {field}/{env.NUM_FIELDS}, life_point: {life_point}, weather: {env.WEATHER_DICT[weather]}:- action: {env.ACTION_DICT[np.argmax(pol)]}")

In [7]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    done = False
    state = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not done:
        enc_state = env.encode(sum(state["field_state"]), state["life_points"], state["weather_type"])
        print(enc_state)
        action = np.argmax(optimal_policy[enc_state])
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, done, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Alive?: {not done}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [1 1 0 0 0]
--Current life: 2
--Type of Weather: Rainy

33
Action to take: Forage
Chance to find food: 0.37
Reward?: -2
is Alive?: False

Current game state:-
--Days left: 5
--State of Field: [0 1 0 0 1]
--Current life: 0
--Type of Weather: Rainy

----------
Episode #2
Initial state:-
--Days left: 6
--State of Field: [1 1 1 1 0]
--Current life: 3
--Type of Weather: Clear

62
Action to take: Forage
Chance to find food: 0.37
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 5
--State of Field: [1 1 0 1 1]
--Current life: 4
--Type of Weather: Clear

64
Action to take: Forage
Chance to find food: 0.95
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 4
--State of Field: [1 1 0 0 1]
--Current life: 5
--Type of Weather: Clear

52
Action to take: Forage
Chance to find food: 0.73
Reward?: -2
is Alive?: True

Current game state:-
--Days left: 3
--State of Field: [0 0 1 0 0]
--Current life: 3
--Type of Weather: R

In [8]:
num_episodes = 1000
episode_reward = []

for episode in range(num_episodes):
    done = False
    state = env.reset()

    total_reward = 0

    while not done:
        enc_state = env.encode(sum(state["field_state"]), state["life_points"], state["weather_type"])
        action = np.argmax(optimal_policy[enc_state])
        state, reward, done, info = env.step(action)

        total_reward += reward

    episode_reward.append(total_reward)

print(f"mean_reward:{np.mean(episode_reward):.2f} +/- {np.std(episode_reward):.2f}")


mean_reward:-2.75 +/- 1.61


In [9]:
model = DQN('MultiInputPolicy', env, exploration_fraction=0.3).learn(total_timesteps=int(2e5))

In [10]:
num_episodes = 2

for episode in range(num_episodes):
    print(f"Episode #{episode+1}")
    print("=" * 10)

    done = False
    state = env.reset()

    print("Initial state:-")
    env.render()
    print()

    while not done:
        action, _ = model.predict(state)
        print(f"Action to take: {env.ACTION_DICT[action]}")
        state, reward, done, info = env.step(action)
        print(f'Chance to find food: {info["chance"]:.2f}')
        print(f"Reward?: {reward}")
        print(f"is Alive?: {not done}")
        print()
        print("Current game state:-")
        env.render()
        print()

    print("-" * 10)

Episode #1
Initial state:-
--Days left: 6
--State of Field: [1 0 1 1 1]
--Current life: 5
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: True

Current game state:-
--Days left: 5
--State of Field: [1 0 0 0 0]
--Current life: 4
--Type of Weather: Rainy

Action to take: Forage
Chance to find food: 0.73
Reward?: 1
is Alive?: True

Current game state:-
--Days left: 4
--State of Field: [0 0 1 0 1]
--Current life: 5
--Type of Weather: Rainy

Action to take: Forage
Chance to find food: 0.16
Reward?: -2
is Alive?: True

Current game state:-
--Days left: 3
--State of Field: [1 1 0 0 1]
--Current life: 3
--Type of Weather: Rainy

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: True

Current game state:-
--Days left: 2
--State of Field: [1 1 1 1 0]
--Current life: 2
--Type of Weather: Clear

Action to take: Wait
Chance to find food: 0.00
Reward?: -1
is Alive?: True

Current game state:-
--Days left: 1
--State of Field: [1 1 1 

In [ ]:
# for idx in range(num_states):
#     field, life_point, weather = env.decode(idx)
#     field_state = np.zeros(env.NUM_FIELDS, dtype=int)
#     field_state[:field] = 1
#     state = {
#         "field_state": field_state,
#         "life_points": life_point,
#         "weather_type": weather
#     }

#     action, _ = model.predict(state, deterministic=True)
#     print(f"field: {field}/{env.NUM_FIELDS}, life_point: {life_point}, weather: {env.WEATHER_DICT[weather]}:- action: {env.ACTION_DICT[action]}")

In [13]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1000)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:-2.71 +/- 1.66
